In [2]:
import requests

In [3]:
from getpass import getpass

In [4]:
token = getpass('Your ONC API Token:')

Your ONC API Token: ····································


In [ ]:
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}

{
       "type": "Feature",
       "bbox": [-10.0, -10.0, 10.0, 10.0],
       "geometry": {
           "type": "Polygon",
           "coordinates": [
               [
                   [-10.0, -10.0],
                   [10.0, -10.0],
                   [10.0, 10.0],
                   [-10.0, -10.0]
               ]
           ]
       }
       //...
   }

{
       "type": "Feature",
       "bbox": [-10.0, -10.0, 10.0, 10.0],
       "geometry": {
           "type": "Polygon",
           "coordinates": [
               [
                   [-10.0, -10.0],
                   [10.0, -10.0],
                   [10.0, 10.0],
                   [-10.0, -10.0]
               ]
           ]
       }
       //...
   }

In [45]:
# https://geojson.org/
import json

class GeoJsonObj(object):
    def __init__(self, obj_type=None):
        self.json_obj={'type': obj_type}
        
    def __str__(self):
        return str(self.json_obj)
    
    def __repr__(self):
        return self.__str__()
    
    def to_json(self):
        return json.dumps(self.json_obj, sort_keys=False, indent=2)
    
class Point(GeoJsonObj):
    def __init__(self, longitude, latitude):
            GeoJsonObj.__init__(self, obj_type='Point')
            self.json_obj['coordinates'] = [longitude, latitude]
            
class Feature(GeoJsonObj):
    def __init__(self, geometry=None, properties=None):
            GeoJsonObj.__init__(self, obj_type='Feature')
            self.json_obj['geometry'] = geometry.json_obj
            self.json_obj['properties'] = properties
            
class FeatureCollection(GeoJsonObj):
    def __init__(self, features=[]):
            GeoJsonObj.__init__(self, obj_type='FeatureCollection')
            self.features = list()
            self.json_obj['features'] = self.features
            for feature in features:
                self.append(feature)
            
    def append(self, feature):
        self.features.append(feature.json_obj)
            
    

In [5]:
r = requests.get('https://data.oceannetworks.ca/EarthquakeCatalogService', params={'operation': 1, 'token': token})

In [7]:
r.json()

{'payload': [{'earthquakeId': 1,
   'latitude': 46.28,
   'longitude': -126.88,
   'magnitude': 3.95,
   'nrcanDetected': False,
   'oncDetected': True,
   'originTime': '2021-12-09T21:09:31.375Z',
   'usgsDetected': False},
  {'earthquakeId': 2,
   'latitude': 50.077,
   'longitude': -130.147,
   'magnitude': 3.75,
   'nrcanDetected': True,
   'oncDetected': False,
   'originTime': '2021-11-27T21:43:53.000Z',
   'usgsDetected': False},
  {'earthquakeId': 3,
   'latitude': 49.981,
   'longitude': -130.204,
   'magnitude': 4.04,
   'nrcanDetected': True,
   'oncDetected': False,
   'originTime': '2021-11-27T12:16:56.000Z',
   'usgsDetected': False},
  {'earthquakeId': 4,
   'latitude': 50.083,
   'longitude': -130.093,
   'magnitude': 3.5,
   'nrcanDetected': True,
   'oncDetected': False,
   'originTime': '2021-11-27T12:05:05.000Z',
   'usgsDetected': False},
  {'earthquakeId': 5,
   'latitude': 50.51,
   'longitude': -130.158,
   'magnitude': 3.63,
   'nrcanDetected': True,
   'oncDet

In [54]:
earthquake_collection = FeatureCollection()

for earthquake in r.json()['payload']:
    p = Point(earthquake['longitude'], earthquake['latitude'])
    url = f'https://data.oceannetworks.ca/EarthquakeDetail?earthquakeId={earthquake["earthquakeId"]}'
    earthquake['url'] = url
    earthquake_collection.append(Feature(p, earthquake))


with open('earthquake_catalog.geojson', 'w') as catalog_file:    
    catalog_file.write(earthquake_collection.to_json())